# Remote Blasting
We are BLASTing remotely against the NR database. In the result we include the subject's accession number (SACC), taxonomic ID (STAXIDS) and description (STITLE).  We use the STR D18S51 having AGAA. We use the SACC to download the sequence via `efetch`. If you see TTCA then exchange start and stop in efetch.

In [ ]:
time blastn -query new_d18s51.fasta -db nr -remote -outfmt "6 qseqid sacc pident length sstart send staxids stitle" -out new.tab -task blastn-short

In [ ]:
primerpair2() { awk -F"\t" '{if($1~/f_/ && $3==100 && $7==9606){f_line[NR]=$1" "$2" "$8; f_acc[NR]=$2; forward[NR]=$5}else if($1~/r_/ && $3==100 && $7==9606){r_line[NR]=$1" "$2" "$8; r_acc[NR]=$2; reverse[NR]=$5}}END{for(x in forward){for(y in reverse){z=forward[x]-reverse[y]; if(z<0){z=z*-1}; if(z<400 && r_acc[y]==f_acc[x]){print forward[x],reverse[y],r_acc[y]"\n"r_line[y]"\n"f_line[x]}}}}' $1; }

In [ ]:
grep 'chromosome 18' new.tab | primerpair2 # try primerpair2 new.tab to see all hits

In [ ]:
efetch -db nuccore -id NG_009361 -format fasta -seq_start 42770 -seq_stop 42465

In [ ]:
primerpair2 new.tab

In [ ]:
efetch -db nuccore -id AK021459 -format fasta -seq_start 273 -seq_stop 342

# Playgound

In [ ]:
pwd

In [ ]:
cd CompBiol3/PrimerBlast

In [ ]:
pwd

In [ ]:
cp hs-chr3.fasta chr3-ref.fasta

In [ ]:
# for i in chr3*fasta; do DB=$(basename $i .fasta); echo $DB; makeblastdb -in $i -dbtype nucl -out $DB -logfile blast.log;  blastn -query D3S1358.primers -db $DB -out d3s1358-vs-$DB.tab -task blastn-short -outfmt "6 qseqid length pident sstart send sstrand; awk '$3==100 && $4>=19{print $1, $9, $10}' d3s1358-vs-$DB.tab; done

In [ ]:
for i in chr3*fasta; do DB=$(basename $i .fasta); echo $DB; makeblastdb -in $i -dbtype nucl -out $DB -logfile blast.log;  blastn -query D3S1358.primers -db $DB -task blastn-short -outfmt "6 qseqid length pident sstart send sstrand" | awk '$3==100 && $2>=19{print $1,$4,$5,$6}'; done

In [ ]:
primerpair() { awk '$0~/chr/{id=$0}{if($1~/f-/){forward[NR]=$3; frange[NR]=$0; fid[NR]=id}else{reverse[NR]=$3; rrange[NR]=$0}}END{for(x in forward){for(y in reverse){z=forward[x]-reverse[y]; if(z<0){z=z*-1}; if(z<400){print fid[x],frange[x],rrange[y]}}}}' $1; }

In [ ]:
for i in chr3*fasta; do DB=$(basename $i .fasta); echo $DB; makeblastdb -in $i -dbtype nucl -out $DB -logfile blast.log;  blastn -query D3S1358.primers -db $DB -task blastn-short -outfmt "6 qseqid length pident sstart send sstrand" | awk '$3==100 && $2>=19{print $1,$4,$5,$6}'; done | primerpair

In [ ]:
rm d3-chr3.fasta

In [ ]:
blastdbcmd -db chr3-ashkenazi -entry all -range 45549893-45550023 -strand plus -line_length 70 | tee -a d3-chr3.fasta

In [ ]:
blastdbcmd -db chr3-japanese -entry all -range 45580408-45580538 -strand plus -line_length 70 | tee -a d3-chr3.fasta

In [ ]:
blastdbcmd -db chr3-pongo -entry all -range 101038273-101038396 -strand minus -line_length 65 | tee -a d3-chr3.fasta

In [ ]:
blastdbcmd -db chr3-ref -entry all -range 45540713-45540843 -strand plus -line_length 65 | tee -a d3-chr3.fasta

In [ ]:
blastdbcmd -db chr3-venter -entry all -range 52359822-52359952 -strand plus -line_length 65 | tee -a d3-chr3.fasta

In [ ]:
clustalo --in d3-chr3.fasta --guidetree-out=clustalo.dnd --force

In [ ]:
njplot clustalo.dnd # opens a new window

# Remote Batch BLAST

## Downloading STR Primer Sequences from NIST

In [ ]:
pandoc 'https://strbase-archive.nist.gov/PP16primers.htm' -f html -t plain | egrep  '(Pair |[ATGC]{10,})' | awk 'NR%3==1{id=$1}NR%3==2{print ">f_"id"\n"$0 > id".str.fasta"}NR%3==0{print ">r_"id"\n"$0 >> id".str.fasta"}'

## Remote Batch BLASTing STR Primers

In [ ]:
time for i in *.str.fasta; do echo $i; blastn -query $i -db nr -remote -max_target_seqs 10 -outfmt "6 qseqid sacc pident length sstart send staxids sstrand" -out $i.tab -task blastn-short; done

## Extracting Amplicons

In [ ]:
cd CompBiol3/PrimerBlast

In [ ]:
head -1 Amelogenin.str.fasta.tab

In [ ]:
for i in test.tab; do echo $i; awk '$3==100 && $4>10 && $7==9606{print $1,$2,$5,$6,$8}' $i | awk -f process.blast.awk; done

In [ ]:
for i in *.str.fasta.tab; do echo $i; awk '$3==100 && $4>10 && $7==9606{print $1,$2,$5,$6,$8}' $i | awk -f process.blast.awk; done 

In [ ]:
for i in *.str.fasta.tab; do echo $i; awk '$3==100 && $4>10 && $7==9606{print $1,$2,$5,$6,$8}' $i | awk -f process.blast.awk; done | awk 'NF==1 {print "==="$0"==="} NF==5 {if($5=="plus") {start=$3;stop=$4} else {start=$4;stop=$3}; cmd="efetch -db nuccore -id "$2" -format fasta -seq_start "start" -seq_stop "stop; while(cmd | getline result) {print result}; close(cmd)}'

In [ ]:
efetch -db nuccore -id AC087436 -format fasta -seq_start 75395 -seq_stop 75618

In [ ]:
efetch -db nuccore -id AC087436 -format fasta -seq_start 75618 -seq_stop 75395

awk '$3==100 && $4>10 && $7==9606{print $1,$2,$5,$6,$8}' test.tab | awk '
{
if($1~/f_/){
    fprimer[NR]=$1; fstart[NR]=$3; fend[NR]=$4; fstrand[NR]=$5; facc[NR]=$2
    }
else{
    rprimer[NR]=$1; rstart[NR]=$3; rend[NR]=$4; rstrand[NR]=$5; racc[NR]=$2
    }
}
END{
for(x in fstart){
    for(y in rstart){
        z=fstart[x]-rstart[y]
        if(z<0){z=z*-1}
        if(z<400 && facc[x]==racc[y]){
            bp[1]=fstart[x];bp[2]=fend[x];bp[3]=rstart[y];bp[4]=rend[y]
            asort(bp)
            print fprimer[x],facc[x],racc[y],bp[1],bp[4]
            }
        }
    }
}